In [1]:
import numpy as np
import os
#os.chdir("C:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\STK_SEARCH")
os.chdir("/rds/general/user/ma11115/home/STK_Search")

In [2]:
#%pip install pytorch-lightning

import os
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl


In [3]:
import numpy as np
slatm_rpr = np.load('data/output/Full_dataset/repr_df_total_new2023_08_20.npy',allow_pickle=True)
slatm_name = np.load('data/output/Full_dataset/names_df_total_new2023_08_20.npy',allow_pickle=True)

In [75]:
import pandas as pd
from src.stk_search import Database_utils
import importlib
importlib.reload(Database_utils)
#%% 
# Load the searched space
df_path = 'data/output/Full_datatset/df_total_new2023_08_20.csv'
df_precursors_path = 'data/output/Prescursor_data/calculation_data_precursor_190923_clean.pkl'#'Data/output/Prescursor_data/calculation_data_precursor_310823_clean.pkl'
df_total, df_precursors = Database_utils.load_data_from_file(df_path, df_precursors_path)


/rds/general/user/ma11115/home/STK_Search/src/stk_search/Database_utils.py:73: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_total = pd.read_csv(df_path)


In [5]:
import numpy as np
slatm_rpr_precursor = np.load('data/output/Prescursor_data/repr_slatm_precursor_310823_clean.pkl.npy',allow_pickle=True)
slatm_name_precursor = np.load('data/output/Prescursor_data/names_slatm_precursor_310823_clean.pkl.npy',allow_pickle=True)
slatm_rpr_precursor = slatm_rpr_precursor[:, ~(slatm_rpr_precursor == slatm_rpr_precursor[0,:]).all(0)]
for i,x in enumerate(slatm_name_precursor):
    x=x.split('/')[1].replace('.xyz','')
    slatm_name_precursor[i]=x

In [6]:
#np.isnan(slatm_rpr).any()
import pandas as pd
slatm_rpr_new=slatm_rpr
print(slatm_rpr_new.shape)
slatm_rpr_new = slatm_rpr_new[:, ~(slatm_rpr_new == slatm_rpr_new[0,:]).all(0)]
print(slatm_rpr_new.shape)

(40975, 40221)
(40975, 14719)


In [7]:
for i in range(6):
    df_total[f'slatm_{i}']=df_total[f'InChIKey_{i}'].apply(lambda x: slatm_rpr_precursor[slatm_name_precursor==x][0])
init_slatm_rpr = np.stack(df_total[f'slatm_{0}'].values)
for i in range(1,6):
    init_slatm_rpr = np.concatenate((init_slatm_rpr,np.stack(df_total[f'slatm_{i}'].values)), axis=1)
init_slatm_rpr.shape

(40975, 25638)

In [8]:
print(slatm_rpr_new.shape)
print(init_slatm_rpr.shape)

(40975, 14719)
(40975, 25638)


In [35]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(25638, 100), nn.ReLU(), nn.Linear(100, 14719))

    def forward(self, x):
        return self.l1(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(14719, 100), nn.ReLU(), nn.Linear(100, 25638))

    def forward(self, x):
        return self.l1(x)

In [37]:
from torchmetrics.functional import accuracy

class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        loss = self._get_preds_loss_accuracy(batch)

        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def validation_step(self, batch, batch_idx):
        '''used for logging metrics'''
        loss = self._get_preds_loss_accuracy(batch)

        # Log loss and metric
        self.log('val_loss', loss)
        return loss
    
    def _get_preds_loss_accuracy(self, batch):
        '''convenience function since train/valid/test steps are similar'''
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x) + F.mse_loss(z, y)
        return loss

In [38]:
from torch.utils.data import Dataset, DataLoader

# convert into PyTorch tensors
X = torch.tensor(init_slatm_rpr, dtype=torch.float32)
y = torch.tensor(slatm_rpr_new, dtype=torch.float32)


# create DataLoader, then take one batch
dataset = list(zip(X,y))
loader = DataLoader(dataset, shuffle=True, batch_size=160,num_workers=6)

In [39]:
dataset = list(zip(X,y))
train_len = int(len(dataset)*0.8)
val_len = len(dataset)-train_len

train_set, val_set = torch.utils.data.random_split(dataset, [train_len, val_len])
batch_size = 100

train_loader = DataLoader(dataset, batch_size=batch_size, 
                                           num_workers=6,shuffle = True)
validation_loader = DataLoader(dataset, batch_size=batch_size,shuffle = False)

In [45]:

# model
from pytorch_lightning.loggers import WandbLogger

import wandb
wandb.login()



autoencoder = LitAutoEncoder(Encoder(), Decoder())
wandb_logger = WandbLogger(project='VAE_slatm',log_model="all")
# train model
trainer = pl.Trainer(logger=wandb_logger,fast_dev_run=False, max_epochs=100,val_check_interval=1.0)
trainer.fit(model=autoencoder, train_dataloaders=train_loader, val_dataloaders=validation_loader)
wandb.finish()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 4.1 M 
1 | decoder | Decoder | 4.1 M 
------------------------------------
8.1 M     Trainable params
0         Non-trainable params
8.1 M     Total params
32.448    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


epoch,▁▁▁▁▂▂▂▂▃▃▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,▃▃▃▃▂▂▂▂▂▂▃▂▃▂▃▂▂▂▂▂▂▂▂▂█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▃▃▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
val_loss,▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
train_loss,0.35332
trainer/global_step,40999
val_loss,0.34925


In [65]:
test_slatm = init_slatm_rpr.reshape(init_slatm_rpr.shape[0],-1,int(init_slatm_rpr.shape[1]/6))
test_slatm[1,1,:]

array([ 0.,  4., 60., ...,  0.,  0.,  0.], dtype=float16)

In [66]:
df_total['slatm_0'][0]

array([ 0.,  4., 60., ...,  0.,  0.,  0.], dtype=float16)

In [121]:
from src.stk_search import Search_algorithm

class Bayesian_Optimisation_slatm(Search_algorithm.Bayesian_Optimisation):
    def prepare_data_for_BO(
        self,
        df_total: pd.DataFrame = [],
        ids_acquired: list = [],
    ):
        df =df_total.loc[np.array(ids_acquired)]

        X_explored_BO = torch.tensor(
            list(df['slatm_opt'].values),
            dtype=torch.float64,
            device=self.device
        )
        X_all = torch.tensor(
            list(df_total['slatm_opt'].values),
            dtype=torch.float64,
            device=self.device
        )
        # normalise the data
        X_explored_max = X_all.max(axis=0).values
        X_explored_min = X_all.min(axis=0).values
        X_explored_BO = (X_explored_BO - X_explored_min) / (
           X_explored_max- X_explored_min
        )

        # limit the dataframe to only the numeric data
        y_explored_BO = torch.tensor(
            df['target'].values, dtype=torch.float64, device=self.device
        )
        # normalise the data
        if y_explored_BO.std() != 0:
            y_explored_BO_norm = (
                y_explored_BO - y_explored_BO.mean()
            ) / y_explored_BO.std()
        else:
            y_explored_BO_norm = y_explored_BO
        y_explored_BO_norm = y_explored_BO_norm.reshape(-1, 1)  # for the GP
        # set up acquisition function
        return X_explored_BO, y_explored_BO_norm


In [160]:
BO = Bayesian_Optimisation_slatm()
BO.device = 'cuda'
ids_acquired=BO.initial_suggestion(df_total, 100)
X_explored_BO, y_explored_BO_norm = BO.prepare_data_for_BO(df_total, ids_acquired=ids_acquired)
BO.train_model(X_explored_BO, y_explored_BO_norm)

/rds/general/user/ma11115/home/STK_Search/src/stk_search/Search_algorithm.py:267: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_train, dtype=torch.float64, device=self.device),


In [180]:
from botorch.acquisition import ExpectedImprovement

best_value = y_explored_BO_norm.max()
EI = ExpectedImprovement(model=BO.model, best_f=best_value)
from botorch.optim import optimize_acqf
d =X_explored_BO.shape[1]
new_point_analytic, _ = optimize_acqf(
    acq_function=EI,
    bounds=torch.tensor([[0.0] * d, [1.0] * d]),
    q=1,
    num_restarts=20,
    raw_samples=100,
    options={},
)
from scipy.spatial import distance
new_element = autoencoder.decoder(new_point_analytic)
def get_closest_element(slatm_rpr_precursor, new_element,frag_num):
# get element in slatm_rpr_precursor that is closest to frag_0
    frag = new_element.reshape(6,-1)[frag_num].detach().numpy()
    dist = np.zeros(slatm_rpr_precursor.shape[0])
    for i in range(slatm_rpr_precursor.shape[0]):
        dist[i] = distance.euclidean(slatm_rpr_precursor[i], frag)   
    return dist.argmin(), dist.min(),dist
for i in range(6):
    print(get_closest_element(slatm_rpr_precursor, new_element,i))


(292, 20.636728286743164)
(292, 20.41227149963379)
(292, 18.439838409423828)
(292, 19.606712341308594)
(292, 20.172183990478516)
(292, 21.01972770690918)


In [201]:
new_point_analytic = X_explored_BO[y_explored_BO_norm.argmax()].to('cpu').float()
new_element = autoencoder.decoder(new_point_analytic)
def get_closest_element(slatm_rpr_precursor, new_element,frag_num):
# get element in slatm_rpr_precursor that is closest to frag_0
    frag = new_element.reshape(6,-1)[frag_num].detach().numpy()
    dist = np.zeros(slatm_rpr_precursor.shape[0])
    for i in range(slatm_rpr_precursor.shape[0]):
        dist[i] = distance.euclidean(slatm_rpr_precursor[i], frag)   
    return dist.argmin(), dist.min()
for i in range(6):
    print(get_closest_element(slatm_rpr_precursor, new_element,i))

(292, 21.22296142578125)
(292, 21.221240997314453)
(292, 19.390853881835938)
(292, 20.469345092773438)
(292, 21.092561721801758)
(292, 21.724645614624023)


In [202]:

frag = new_element.reshape(6,-1)[0].detach().numpy()
frag.shape

(4273,)

In [207]:
distance.euclidean(new_element[:4273].detach().numpy(),frag)

0.0

In [189]:
new_point_analytic.dtype

torch.float32

8.744457244873047

(292, 20.02475357055664)